# 根据 url 实现对新闻内容的提炼

## 最简原型

In [1]:
%%time

from llama_index.core.tools import FunctionTool
from llama_index.llms.openai import OpenAI
from llama_index.core.agent import ReActAgent

CPU times: user 2.79 s, sys: 405 ms, total: 3.2 s
Wall time: 2.92 s


In [10]:
%%time

from llama_index.llms.openai_like import OpenAILike
from llama_index.core import Settings

llm = OpenAILike(model="qwen2-7b-6k", 
                 api_base="http://192.168.0.72:3000/v1", 
                 api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
                 is_chat_model=True,
                 temperature=0.1
                )

llm.complete(prompt="根据 {query_str} 访问并返回内容，然后提炼内容作为回答。")

Settings.llm =llm

CPU times: user 63.4 ms, sys: 0 ns, total: 63.4 ms
Wall time: 2.29 s


In [13]:
%%time

content="""
目前巴基斯坦面临着经济和政治上的不少问题，在这样一个背景下，我们总理访华肯定希望和中国加强在各个领域，尤其是经济方面的合作。中巴传统友谊经过了长时间的一代又一代人的历史考验，但当前巴基斯坦需要更多外部帮助，比如说我们在经济上需要来自海外投资，现在中国正好有这个条件。习近平主席一直鼓励中国企业走出去，给有志于海外市场拓展的企业一些补助，或者其他方面的优惠政策。巴基斯坦非常欢迎中国企业来我们国内发展。
"""

# define sample Tool
def get_url_content(url:str) -> str:
    """根据 url 访问并返回内容"""
    return f"新闻内容是{content}"

get_url_content_tool = FunctionTool.from_defaults(fn=get_url_content)

agent = ReActAgent.from_tools([get_url_content_tool], llm=llm, verbose=True)

CPU times: user 493 µs, sys: 0 ns, total: 493 µs
Wall time: 497 µs


In [14]:
%%time

stream_response=agent.stream_chat("https://www.guancha.cn/ZamirAhmedAwan/2024_06_08_737417.shtml")
stream_response.print_response_stream()

Thought: The user has provided a URL. I need to use the tool 'get_url_content' to fetch the content from this URL.
Action: get_url_content
Action Input: {'url': 'https://www.guancha.cn/ZamirAhmedAwan/2024_06_08_737417.shtml'}
Observation: 新闻内容是
目前巴基斯坦面临着经济和政治上的不少问题，在这样一个背景下，我们总理访华肯定希望和中国加强在各个领域，尤其是经济方面的合作。中巴传统友谊经过了长时间的一代又一代人的历史考验，但当前巴基斯坦需要更多外部帮助，比如说我们在经济上需要来自海外投资，现在中国正好有这个条件。习近平主席一直鼓励中国企业走出去，给有志于海外市场拓展的企业一些补助，或者其他方面的优惠政策。巴基斯坦非常欢迎中国企业来我们国内发展。

 这段新闻提到目前巴基斯坦在经济和政治上面临不少问题，在这样的背景下，总理访问中国旨在加强两国在各个领域的合作，尤其是经济方面。中巴之间的传统友谊经过了长时间的考验，但当前巴基斯坦需要更多外部帮助，包括经济上的投资。中国领导人鼓励中国企业走出去，并为有志于海外市场拓展的企业提供补助或其他优惠政策。巴基斯坦非常欢迎中国的投资和发展。CPU times: user 1.44 s, sys: 371 ms, total: 1.81 s
Wall time: 4.61 s


## 完整功能的原型

In [41]:
%%time

from llama_index.llms.openai_like import OpenAILike
from llama_index.core import Settings

llm = OpenAILike(model="qwen2-7b-6k", 
                 api_base="http://192.168.0.72:3000/v1", 
                 api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
                 is_chat_model=True,
                 temperature=0.1
                )

llm.complete(prompt=
             """
             回答要求用中文。
             根据 {query_str} 访问并返回内容。
             """
             )

Settings.llm =llm

CPU times: user 52.4 ms, sys: 456 µs, total: 52.9 ms
Wall time: 3.7 s


In [42]:
%%time

from gne import GeneralNewsExtractor
import requests

def get_url_content(url:str) -> str:
    """根据 url 访问并返回内容"""
    response = requests.get(url)
    html=response.text

    extractor = GeneralNewsExtractor()
    result = extractor.extract(html, noise_node_list=['//div[@class="comment-list"]'])
    
    return f"新闻内容是: {result}。\n\n根据新闻内容回答，文章标题，文章作者，文章发表时间，以及文章的主要内容。"

get_url_content_tool = FunctionTool.from_defaults(fn=get_url_content)

agent = ReActAgent.from_tools([get_url_content_tool], llm=llm, verbose=True)

CPU times: user 558 µs, sys: 111 µs, total: 669 µs
Wall time: 674 µs


In [43]:
%%time

stream_response=agent.stream_chat("https://www.guancha.cn/ZamirAhmedAwan/2024_06_08_737417.shtml")
stream_response.print_response_stream()

Thought: The user has provided a URL. I need to use the 'get_url_content' tool to fetch the content of this webpage.
Action: get_url_content
Action Input: {'url': 'https://www.guancha.cn/ZamirAhmedAwan/2024_06_08_737417.shtml'}
Observation: 新闻内容是: {'title': '扎米尔·艾哈迈德·阿旺：对接中国企业落地，巴基斯坦有三大优势', 'author': '武守哲', 'publish_time': '2024-06-08 09:02:02', 'content': '观察者网：今年巴基斯坦新政府成立之后，夏巴兹总理首次访华。从巴基斯坦方面来看，对这次访华有哪些期待？\n扎米尔·艾哈迈德·阿旺\n：\n目前巴基斯坦面临着经济和政治上的不少问题，在这样一个背景下，我们总理访华肯定希望和中国加强在各个领域，尤其是经济方面的合作。中巴传统友谊经过了长时间的一代又一代人的历史考验，但当前巴基斯坦需要更多外部帮助，比如说我们在经济上需要来自海外投资，现在中国正好有这个条件。习近平主席一直鼓励中国企业走出去，给有志于海外市场拓展的企业一些补助，或者其他方面的优惠政策。巴基斯坦非常欢迎中国企业来我们国内发展。\n巴基斯坦发展经济的潜力巨大，从以下几个方面可以体现出来。\n首先，我们的劳动力市场很广阔。巴基斯坦总体人口结构很年轻，70%的人口都在40岁以下；\n其次，巴基斯坦劳动力成本相对较低，跟中国相比，至少便宜40%，中国企业在巴基斯坦落地发展，劳动力的性价比对企业利润是一种保证；\n再者，巴基斯坦有丰富的原材料，而且也比较便宜，这是中国制造业实体所非常看中的。\n除此之外，深圳在6月5日召开了巴基斯坦中国商务会议。召开这一会议主要目的就是吸引中国企业投资。会议主旨和议题目前特别指定了几个细分赛道，都和高科技产业相关。当然，经济合作的前提是双边牢固的外交关系，这是双方互惠互利的前提。巴基斯坦政坛高层和习近平主席、李强总理都有过频繁的会面，双方会就一些双方关注的问题会进行深入探讨。在重大安全和领土主权问题上，如中国台湾问题，新疆问题，香港

In [44]:
%%time

stream_response=agent.stream_chat("根据 https://www.guancha.cn/HasanUnal/2024_06_08_737418.shtml 给出文章内容要点")
stream_response.print_response_stream()

Thought: The current language of the user is: Chinese. I need to use a tool to help me answer the question.
Action: get_url_content
Action Input: {'url': 'https://www.guancha.cn/HasanUnal/2024_06_08_737418.shtml'}
Observation: 新闻内容是: {'title': '余纳尔：多极化将削弱西方集团权力，这是土耳其的机遇', 'author': '郭涵', 'publish_time': '2024-06-08 09:10:06', 'content': '【文/余纳尔（Hasan Ünal），翻译/观察者网 郭涵】\n6月3日至5日正式访华期间，土耳其外交部长费丹（Hakan Fidan）概述了安卡拉对待多极世界秩序的态度，强调土耳其渴望与中国深化关系。费丹表示土耳其希望加入金砖国家组织，并批评了西方集团基于所谓“产能过剩”的借口在亚太地区遏制中国的努力。“产能过剩论”是近期美国高官多次抛出的一段古怪言论。\n除了推动土中两国在经贸领域的进一步合作外，费丹主要谈到了一个正在兴起的多极化格局，在土耳其外长看来，这事实上是（也应当被视作）一个势力均衡的自然过程。西方殖民主义列强在20世纪建立的世界体系已经无法在21世纪继续维持。他批评了以延缓中国经济发展、破坏中国稳定为目标，通过各种方式遏制中国的努力。\n费丹表示，安卡拉主张，从美国领导下的单极国际体系向多极化世界的过渡是一个自然过程，因此土耳其无法接受外国针对中国发起的舆论战，特别是外国就台湾问题以战争相威胁的做法。土耳其外长指出，地球上有充足的资源供世界各国分享，全世界事实上需要一个公正平等的国际秩序，对所有主权国家均一视同仁。\n中共中央政治局委员、外交部长王毅同土耳其外长费丹在北京举行会谈。 图自：中国外交部网站\n中国领导人在近年来提出的一系列倡议中详细阐述了他对国际事务的看法，包括《全球文明倡议》、《全球安全倡议》和《全球发展倡议》。而土耳其外长费丹关于世界体系转型的主张，在许多地方形成呼应。值得注意的是，土耳其自1952年以来就被视作北约长期可靠的成员国，也是从1950年代起、整个冷战期间

## 8k上下文测试

In [45]:
%%time

from llama_index.llms.openai_like import OpenAILike
from llama_index.core import Settings

llm = OpenAILike(model="qwen2-7b-8k", 
                 api_base="http://192.168.0.72:3000/v1", 
                 api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
                 is_chat_model=True,
                 temperature=0.1
                )

llm.complete(prompt=
             """
             回答要求用中文。
             根据 {query_str} 访问并返回内容。
             """
             )

Settings.llm =llm

CPU times: user 51.4 ms, sys: 3.24 ms, total: 54.7 ms
Wall time: 7.27 s


In [48]:
%%time

from gne import GeneralNewsExtractor
import requests

def get_url_content(url:str) -> str:
    """根据 url 访问并返回内容"""
    response = requests.get(url)
    html=response.text

    extractor = GeneralNewsExtractor()
    result = extractor.extract(html, noise_node_list=['//div[@class="comment-list"]'])
    
    return f"新闻内容是: {result}。\n\n根据新闻内容回答。\n\n回答仿照下面方式：\n\n这篇文章的标题是教育部教育考试院发布高考语文全国卷六大创新题型及解析，作者是观察者网，发布时间是2024年6月8日下午3点。文章的主要内容是..."

get_url_content_tool = FunctionTool.from_defaults(fn=get_url_content)

agent = ReActAgent.from_tools([get_url_content_tool], llm=llm, verbose=True)

CPU times: user 547 µs, sys: 0 ns, total: 547 µs
Wall time: 552 µs


In [49]:
%%time

stream_response=agent.stream_chat("https://www.guancha.cn/ZamirAhmedAwan/2024_06_08_737417.shtml")
stream_response.print_response_stream()

Thought: The user has provided a URL. I need to use the 'get_url_content' tool to fetch the content from this URL.
Action: get_url_content
Action Input: {'url': 'https://www.guancha.cn/ZamirAhmedAwan/2024_06_08_737417.shtml'}
Observation: 新闻内容是: {'title': '扎米尔·艾哈迈德·阿旺：对接中国企业落地，巴基斯坦有三大优势', 'author': '武守哲', 'publish_time': '2024-06-08 09:02:02', 'content': '观察者网：今年巴基斯坦新政府成立之后，夏巴兹总理首次访华。从巴基斯坦方面来看，对这次访华有哪些期待？\n扎米尔·艾哈迈德·阿旺\n：\n目前巴基斯坦面临着经济和政治上的不少问题，在这样一个背景下，我们总理访华肯定希望和中国加强在各个领域，尤其是经济方面的合作。中巴传统友谊经过了长时间的一代又一代人的历史考验，但当前巴基斯坦需要更多外部帮助，比如说我们在经济上需要来自海外投资，现在中国正好有这个条件。习近平主席一直鼓励中国企业走出去，给有志于海外市场拓展的企业一些补助，或者其他方面的优惠政策。巴基斯坦非常欢迎中国企业来我们国内发展。\n巴基斯坦发展经济的潜力巨大，从以下几个方面可以体现出来。\n首先，我们的劳动力市场很广阔。巴基斯坦总体人口结构很年轻，70%的人口都在40岁以下；\n其次，巴基斯坦劳动力成本相对较低，跟中国相比，至少便宜40%，中国企业在巴基斯坦落地发展，劳动力的性价比对企业利润是一种保证；\n再者，巴基斯坦有丰富的原材料，而且也比较便宜，这是中国制造业实体所非常看中的。\n除此之外，深圳在6月5日召开了巴基斯坦中国商务会议。召开这一会议主要目的就是吸引中国企业投资。会议主旨和议题目前特别指定了几个细分赛道，都和高科技产业相关。当然，经济合作的前提是双边牢固的外交关系，这是双方互惠互利的前提。巴基斯坦政坛高层和习近平主席、李强总理都有过频繁的会面，双方会就一些双方关注的问题会进行深入探讨。在重大安全和领土主权问题上，如中国台湾问题，新疆问题，香港问题

## 10k上下文测试 - 超过8k会报错，Reached max iterations.

In [2]:
%%time

from llama_index.llms.openai_like import OpenAILike
from llama_index.core import Settings

llm = OpenAILike(model="qwen2-7b-10k", 
                 api_base="http://192.168.0.72:3000/v1", 
                 api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
                 is_chat_model=True,
                 temperature=0.1
                )

llm.complete(prompt=
             """
             回答要求用中文。
             根据 {query_str} 访问并返回内容。
             """
             )

Settings.llm =llm

CPU times: user 181 ms, sys: 7.53 ms, total: 189 ms
Wall time: 4.31 s


In [3]:
%%time

from gne import GeneralNewsExtractor
import requests

def get_url_content(url:str) -> str:
    """根据 url 访问并返回内容"""
    response = requests.get(url)
    html=response.text

    extractor = GeneralNewsExtractor()
    result = extractor.extract(html, noise_node_list=['//div[@class="comment-list"]'])
    
    return f"新闻内容是: {result}。\n\n根据新闻内容回答。\n\n回答仿照下面方式：\n\n这篇文章的标题是教育部教育考试院发布高考语文全国卷六大创新题型及解析，作者是观察者网，发布时间是2024年6月8日下午3点。文章的主要内容是..."

get_url_content_tool = FunctionTool.from_defaults(fn=get_url_content)

agent = ReActAgent.from_tools([get_url_content_tool], llm=llm, verbose=True)

CPU times: user 250 ms, sys: 20 ms, total: 270 ms
Wall time: 270 ms


In [7]:
%%time

stream_response=agent.stream_chat("https://www.guancha.cn/internation/2024_06_08_737454_s.shtml")
stream_response.print_response_stream()

Thought: The current language of the user is: Chinese. I need to use a tool to help me answer the question.
Action: get_url_content
Action Input: {'url': 'https://www.guancha.cn/internation/2024_06_08_737454_s.shtml'}
Observation: 新闻内容是: {'title': '“塞尔维亚正成为中国附庸”？武契奇反驳：塞是自由国家，中国尊重我们', 'author': '阮佳琪', 'publish_time': '2024-06-08 18:03:49', 'content': '【文/观察者网 阮佳琪】据俄新社7日报道，当地时间周五，由中方研制的匈塞铁路高速动车组首次在塞尔维亚亮相，塞总统武契奇在参观时谈及与中方的合作项目，并且驳斥了关于“塞尔维亚正成为中国附庸”的无端指责。\n综合塞尔维亚《政治报》报道，当时有塞媒提问，如何看待有政治对手声称塞尔维亚正在成为中国的“附庸”。武契奇对这一谬论反驳道：“我不知道你最近有没有注意到，我想我已经老了，我没有力气与胡说八道的人争论。塞尔维亚和塞尔维亚人民从来就不是附庸，也永远不会带来对任何人实行附庸政策的人。我们是一个自由的国家，我们的中国兄弟尊重这一事实，这就是我们两国关系如此之好的原因之一。”\n据塞媒报道，当天在中国驻塞大使李明的陪同下，武契奇与一些塞尔维亚官员在贝尔格莱德泽蒙车辆段，参观匈塞铁路中国高速动车组。\n（武契奇Instagram账号）\n武契奇表示，2023年塞尔维亚同中方签署购买了5列高速列车协议，成为欧洲第一个购买并接收中国高速列车的国家。当天众人参观的正是这批订单中的首列动车，预计最后一列将于2025年12月底交付。\n据他介绍，匈塞铁路塞尔维亚段全线贯通后，从首都贝尔格莱德到塞匈边境城市苏博蒂察仅需1小时10分钟。塞尔维亚预计将从今年9月开始对该铁路段进行测试，塞民众最晚可以在12月体验上这列动车。\n“列车每两个小时一班。与之前的列车不同，这列车分为一等座和二等座。一等座有45个座位，二等座有197个座位。一等座看起来简直（好得）不真实”，武契奇说。\n塞《信使报》称，武契奇走

In [5]:
%%time

stream_response=agent.stream_chat("https://user.guancha.cn/main/content?id=1246861")
stream_response.print_response_stream()

Thought: The current language of the user is: Chinese. I need to use a tool to help me answer the question.
Action: get_url_content
Action Input: {'url': 'https://user.guancha.cn/main/content?id=1246861'}
Observation: 新闻内容是: {'title': '印学者锐评“莫迪经济学”, 反倒戳中发展中国家的一大软肋 | 文化纵横', 'author': '打开观察者网APP', 'publish_time': '2021/07/29', 'content': 'Pranab Bardhan\n（美）加州大学伯克利分校\n\u200b李半田, 谈行藏（译）\n欧亚系统科学研究会\n“新”印度：政治经济诊断（下）\n第二，中央政府利用党派任命地方官员以干涉地方行政职能，任命多是违反程序正当性的。\n中央政府往往绕过邦政府，直接向地区一级的公职人员下达命令，例如中央资助计划的固定执行事务上。最近，印度中央政府提议改变印度行政部门的规则，以便进一步集中官僚机构。\n第三，中央政府通过废除计划委员会\n（Planning Commission）\n严重侵犯了各邦的财政结构，\n该委员会的大部分拨款授权已移交给财政部，将第十五届财政委员会的职权调向了更加集权的方向（尽管委员会已经巧妙地试图绕过一些变化）。它强加了各种各样的地方税收和附加费用，缩减了与各邦分配的收入池，并推迟了勉强分配给各邦的商品和服务税（GST）收入——为此，各州同意放弃部分财政自主权，希望获得更大的补偿。\n第四，在法律与秩序、农业和劳工立法方面，中央政府几乎没有与各邦进行任何磋商就强行通过了独断性法律。\n对少数民族和持不同政见者肆意使用《非法活动防止法》（Unlawful Activities Prevention Act），利用中央调查机关对反对派政治人士进行调查，这都是对惩罚性权力的任意集中和对国家秩序的破坏。\n其结果是，政府所谓的“合作联邦制”\n（cooperative federalism）\n一直是一种欺骗，中央政府与邦之间的关系充满了混乱和不信任，这也反映在对新冠疫情的管理不善上。

In [ ]:
%%time

# 上下文超过8k在4600ti/16G下报错：Reached max iterations.

stream_response=agent.stream_chat("https://user.guancha.cn/main/content?id=1233873")
stream_response.print_response_stream()
# https://user.guancha.cn/main/content?id=1233873